In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
import os
import joblib
from sklearn.metrics import roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV,StratifiedKFold
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score
from sklearn.metrics import classification_report,f1_score #, plot_confusion_matrix
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import learning_curve
from mlxtend.plotting import plot_decision_regions

In [ ]:
df = pd.read_csv('/kaggle/input/cervical-cancer-dataset/cervical-cancer_csv.csv')
df.head(10)

In [ ]:
#Checks shape
print(df.shape)

In [ ]:
#Checks null values
df = df.replace('?', np.NaN)
print(df.isnull().sum())
print('Total number of null values: ', df.isnull().sum().sum())
plt.figure(figsize=(10,10))
np.round(df.isnull().sum()/df.shape[0]*100).sort_values().plot(\
 kind='barh')

In [ ]:
#Drops two columns of STDs
df.drop(['STDs: Time since first diagnosis','STDs: Time since last diagnosis'],inplace=True,axis=1)


In [ ]:
numerical_df = ['Age', 'Number of sexual partners', \
'First sexual intercourse','Num of pregnancies', \
'Smokes (years)', 'Smokes (packs/year)',\
'Hormonal Contraceptives (years)','IUD (years)',\
'STDs (number)']
categorical_df = ['Smokes','Hormonal Contraceptives',\
'IUD','STDs','STDs:condylomatosis',\
'STDs:cervical condylomatosis',\
'STDs:vaginal condylomatosis',\
'STDs:vulvo-perineal condylomatosis',
'STDs:syphilis',\
'STDs:pelvic inflammatory disease', \
'STDs:genital herpes','STDs:molluscum contagiosum', \
'STDs:AIDS', 'STDs:HIV','STDs:Hepatitis B',
'STDs:HPV', \
'STDs: Number of diagnosis','Dx:Cancer', 'Dx:CIN',\
'Dx:HPV', 'Dx', 'Hinselmann', 'Schiller','Citology',
'Biopsy']


In [ ]:
#Fills the missing values of numeric data columns with mean of the column data.
for feature in numerical_df:
    print(feature,'',df[feature].apply(pd.to_numeric,errors='coerce').mean())
    feature_mean = round(df[feature].apply(pd.to_numeric,  errors='coerce').mean(),1)
    df[feature] = df[feature].fillna(feature_mean)
for feature in categorical_df: 
    df[feature] = df[feature].apply(pd.to_numeric, errors='coerce').fillna(1.0)


In [ ]:
#Effect 'Hormonal Contraceptives'
corrmat = df.corr()
k = 15 #number of variables for heatmap
cols = corrmat.nlargest(k, 'Hormonal Contraceptives')['Hormonal Contraceptives'].index
cm = df[cols].corr()
plt.figure(figsize=(25,20))
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, cmap='Set1',annot=True,vmin=0,vmax =1, square=True, fmt='.2f',  annot_kws={'size': 10}, yticklabels = cols.values, xticklabels = cols.values)
plt.show()



In [ ]:
#Effect 'IUD'
corrmat = df.corr()
k = 15 #number of variables for heatmap
cols = corrmat.nlargest(k, 'IUD')['IUD'].index
cm = df[cols].corr()
plt.figure(figsize=(25,20))
sns.set(font_scale=1.25)
hm = sns.heatmap(cm,cmap = 'rainbow', cbar=True, annot=True,vmin=0,vmax =1, square=True,fmt='.2f', annot_kws={'size': 10},yticklabels = cols.values, \
 xticklabels = cols.values)
plt.show()


In [ ]:
#Effect 'STDs'
corrmat = df.corr()
k = 15 #number of variables for heatmap
cols = corrmat.nlargest(k, 'STDs')['STDs'].index
cm = df[cols].corr()
plt.figure(figsize=(25,20))
sns.set(font_scale=1.25)
hm = sns.heatmap(cm,cmap = 'hot', cbar=True,annot=True,vmin=0,vmax =1, square=True,fmt='.2f', annot_kws={'size': 10}, yticklabels = cols.values,\
 xticklabels = cols.values)
plt.show()


In [ ]:
#Correlation Matrix for each feature
corrmat = round(df.corr(), 2)
top_corr_features = corrmat.index
plt.figure(figsize=(25,20))
#plot heat map
g=sns.heatmap(df[top_corr_features].corr(),annot=True,fmt='.2f',cmap="YlGnBu")


In [ ]:
# Defines function to create pie chart and bar plot as subplots
def plot_pie_barchart(df, var, title=''):
    plt.figure(figsize=(25, 10))
    # Pie Chart (Left Subplot)
    plt.subplot(121)
    label_list = list(df[var].value_counts().index)
    colors = sns.color_palette("Set2", len(label_list))
    # Use the 'pastel' color palette
    _, _, autopcts = plt.pie(df[var].value_counts(),autopct="%1.1f%%", colors=colors,startangle=60,labels=label_list, wedgeprops={"linewidth": 2,"edgecolor": "white"},shadow=True, textprops={'fontsize': 20})
    plt.title("Distribution of " + var + " variable " + title, fontsize=25)
    # Extract percentage values from autopcts
    percentage_values = [float(p.get_text().strip('%'))for p in autopcts]
    # Print percentage values
    print("Percentage values:")
    for label, percentage in zip(label_list,percentage_values):
        print(f"{label}: {percentage:.1f}%")
    # Bar Plot (Right Subplot)
    plt.subplot(122)
    ax = df[var].value_counts().plot(kind="barh",
    color=colors, alpha=0.8) # Increase opacity (alpha)
    for i, j in enumerate(df[var].value_counts().values):
        ax.text(.7, i, j, weight="bold", fontsize=20)
        plt.title("Count of " + var + " cases " + title,fontsize=25)
    # Print count values
    value_counts = df[var].value_counts()
    print("Count values:")
    print(value_counts)
    plt.show()

plot_pie_barchart(df,'Biopsy')


The analysis of the "Biopsy" variable using the plot_pie_barchart() function reveals important insights about
the dataset:

Class Imbalance: The distribution of the "Biopsy" variable is highly imbalanced, with
approximately 93.6% of cases having a
negative biopsy result (0) and only about
6.4% having a positive biopsy result (1).
This indicates a significant disparity in the
representation of the two classes, with the
negative class being dominant. Such class
imbalance can impact the performance of
machine learning algorithms, especially
those that are sensitive to class distribution.
Minority Class: The positive biopsy results
(1) represent a relatively small minority of
the dataset. This suggests that the dataset
might be skewed toward negative results,
making it more challenging to accurately
predict and classify positive biopsy cases.
Models trained on imbalanced datasets may
struggle to learn patterns from the minority
class due to its limited representation.
Model Considerations: When building
predictive models using this dataset, it's
important to address the class imbalance
issue. Techniques such as resampling
(oversampling the minority class,
undersampling the majority class), using
appropriate evaluation metrics (precision,
recall, F1-score), and applying algorithms


In [ ]:
plot_pie_barchart(df,'Hinselmann')


In [ ]:
plot_pie_barchart(df,'Schiller')

In [ ]:
 plot_pie_barchart(df,'Citology')


#Plots distribution of number of cases of all other features versus one feature
def others_versus_one_feat(columns, one_feat): 
    ROWS = 7
    COLS = 5
    fig, ax = plt.subplots(ROWS, COLS, figsize=(80,60), constrained_layout=True, facecolor='#fbe7dd')
    for i in range(ROWS):
        ax_index = COLS * i + j
    for j in range(COLS):
        ax_index = COLS * i + j
        if ax_index >= len(columns): 
            break
        g = sns.countplot(data=df,x=columns[ax_index], hue=one_feat, palette='Set2',ax=ax[i, j])
        g.set_xlabel(columns[ax_index], fontsize=50)
        g.set_ylabel('Number of Cases', fontsize=40)
        g.tick_params(axis='both', labelsize=30)
        g.legend(title=one_feat, title_fontsize=50,fontsize=30)
        for p in g.patches:
            g.annotate(format(p.get_height(), '.0f'),(p.get_x() + p.get_width() / 2.,p.get_height()),ha='center', va='center', xytext=(0, 10),weight='bold', fontsize=30,textcoords='offset points')
    plt.show()
#Plots distribution of count of other features versus Biopsy
columns=list(df.columns)
columns.remove('Biopsy')
others_versus_one_feat(columns, "Biopsy")

#Plots distribution of count of other features versus Hinselmann
columns=list(df.columns)
columns.remove('Hinselmann')
others_versus_one_feat(columns, "Hinselmann")


# Features Importance Using Random Forest Classifier

In [ ]:
#Extracts input and output variables
X = df.drop('Biopsy', axis =1)
y = df["Biopsy"]
#Feature Importance using RandomForest Classifier
names = X.columns
rf = RandomForestClassifier()
rf.fit(X, y)
result_rf = pd.DataFrame()
result_rf['Features'] = X.columns
result_rf ['Values'] = rf.feature_importances_
result_rf.sort_values('Values', inplace = True,
ascending = False)
plt.figure(figsize=(25,25))
sns.set_color_codes("pastel")
sns.barplot(x = 'Values',y = 'Features', data=result_rf,color="Blue")
plt.xlabel('Feature Importance', fontsize=30)
plt.ylabel('Feature Labels', fontsize=30)
plt.tick_params(axis='x', labelsize=20)
plt.tick_params(axis='y', labelsize=20)
plt.show()
# Print the feature importance table
print("Feature Importance:")
print(result_rf)


 Conclusion:
Based on the feature importance scores,
"Schiller" and "Hinselmann" are the
most influential features for predicting
the "Biopsy" outcome. It suggests that
the presence or absence of certain
conditions indicated by these features
has a strong impact on the likelihood of
a positive biopsy result.
Other important features, like "Age,"
"Hormonal Contraceptives (years),"
and "First sexual intercourse," also
contribute significantly to the
prediction.
Features with low importance scores
may not contribute significantly to the
prediction and might be candidates for
potential feature selection or further
investigation.

# Features Importance Using Extra Trees Classifier

In [ ]:
#Feature Importance using ExtraTreesClassifier 
model = ExtraTreesClassifier()
model.fit(X, y)
result_et = pd.DataFrame()
result_et['Features'] = X.columns
result_et ['Values'] = model.feature_importances_
result_et.sort_values('Values', inplace=True,ascending =False)
plt.figure(figsize=(25,25))
sns.set_color_codes("pastel")
sns.barplot(x = 'Values',y = 'Features',data=result_et, color="red")
plt.xlabel('Feature Importance', fontsize=30)
plt.ylabel('Feature Labels', fontsize=30)
plt.tick_params(axis='x', labelsize=20)
plt.tick_params(axis='y', labelsize=20)
plt.show() 
# Print the feature importance table
print("Feature Importance:")
print(result_et)


# Features Importance Using RFE

In [ ]:
#Feature Importance using RFE 
from sklearn.feature_selection import RFE
model = LogisticRegression()
# create the RFE model
rfe = RFE(model)
rfe = rfe.fit(X, y)
result_lg = pd.DataFrame()
result_lg['Features'] = X.columns
result_lg ['Ranking'] = rfe.ranking_
result_lg.sort_values('Ranking', inplace=True ,ascending = False)
plt.figure(figsize=(25,25))
sns.set_color_codes("pastel")
sns.barplot(x = 'Ranking',y = 'Features',
data=result_lg, color="orange")
plt.ylabel('Feature Labels', fontsize=30)
plt.tick_params(axis='x', labelsize=20)
plt.tick_params(axis='y', labelsize=20)
plt.show() 
print("Feature Ranking:")
print(result_lg)

A lower rank indicates higher importance, implying
that features with a lower rank are considered more important
according to the RFE ranking

In [ ]:
X = df.drop('Biopsy', axis =1).apply(pd.to_numeric, errors='coerce').astype('float64')
y = df["Biopsy"]
sm = SMOTE(random_state=42)
X,y = sm.fit_resample(X, y.ravel())
#Splits the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X,
y, test_size = 0.2, random_state = 2021, stratify=y) 
X_train_raw = X_train.copy()
X_test_raw = X_test.copy()
y_train_raw = y_train.copy()
y_test_raw = y_test.copy()
X_train_norm = X_train.copy()
X_test_norm = X_test.copy()
y_train_norm = y_train.copy()
y_test_norm = y_test.copy()
norm = MinMaxScaler()
X_train_norm = norm.fit_transform(X_train_norm)
X_test_norm = norm.transform(X_test_norm)
X_train_stand = X_train.copy()
X_test_stand = X_test.copy()
y_train_stand = y_train.copy()
y_test_stand = y_test.copy()
scaler = StandardScaler()
X_train_stand = scaler.fit_transform(X_train_stand)
X_test_stand = scaler.transform(X_test_stand)

In [ ]:
def plot_learning_curve(estimator, title, X, y,axes=None, ylim=None, cv=None, n_jobs=None,train_sizes=np.linspace(.1, 1.0, 5)):
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))
        axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
        axes[0].set_xlabel("Training examples")
        axes[0].set_ylabel("Score")
        train_sizes, train_scores, test_scores, fit_times, _ =learning_curve(estimator, X, y, cv=cv,n_jobs=n_jobs, train_sizes=train_sizes,return_times=True)
        train_scores_mean = np.mean(train_scores,axis=1)
        train_scores_std = np.std(train_scores, axis=1)
        test_scores_mean = np.mean(test_scores, axis=1)
        test_scores_std = np.std(test_scores, axis=1)
        fit_times_mean = np.mean(fit_times, axis=1)
        fit_times_std = np.std(fit_times, axis=1)
# Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r") 
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,test_scores_mean + test_scores_std, alpha=0.1, color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")
    axes[0].legend(loc="best")
# Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, \
    fit_times_mean - fit_times_std,\
    fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")
    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, \
    test_scores_mean - test_scores_std,\
    test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")
    return plt 

In [ ]:
def plot_real_pred_val(Y_test, ypred, name,fc):
    plt.figure(figsize=(25,15))
    acc=accuracy_score(Y_test,ypred)
    plt.scatter(range(len(ypred)),ypred,color="blue",\
    lw=5,label="Predicted")
    plt.scatter(range(len(Y_test)), \
    Y_test,color="red",label="Actual")
    plt.title("Predicted Values vs True Values of " +
    name+" with " + fc + " scaling", \
    fontsize=35)
    plt.xlabel("Accuracy: " + str(round((acc*100),3)) +
    "%", fontsize=25)
    plt.legend(fontsize=20)
    plt.grid(True, alpha=0.75, lw=1, ls='-.')
    plt.show()
    
def plot_cm(Y_test, ypred, name, fc):
    plt.figure(figsize=(25, 15))
    ax = plt.subplot()
    cm = confusion_matrix(Y_test, ypred)
    sns.heatmap(cm, annot=True, linewidth=3,
    linecolor='red', fmt='g', cmap="YlOrBr", annot_kws=
    {"size": 25})
    plt.title(name + ' Confusion Matrix' + " with " + fc
    + " scaling", fontsize=35)
    plt.xlabel('Y predict', fontsize=20)
    plt.ylabel('Y test', fontsize=20)
    ax.xaxis.set_ticklabels(['Biopsy = 0', 'Biopsy = 1'],
    fontsize=25)
    ax.yaxis.set_ticklabels(['Biopsy', 'Biopsy = 1'],
    fontsize=25)
    plt.show()
    return cm

In [ ]:
#Plots ROC
def plot_roc(model,X_test, y_test, title, fc):
    Y_pred_prob = model.predict_proba(X_test)
    Y_pred_prob = Y_pred_prob[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, Y_pred_prob)
    plt.figure(figsize=(25,15))
    plt.plot([0,1],[0,1], color='navy', linestyle='--',linewidth=5)
    plt.plot(fpr,tpr, color='red', linewidth=5)
    plt.xlabel('False Positive Rate', fontsize=25)
    plt.ylabel('True Positive Rate', fontsize=25)
    plt.title('ROC Curve of ' + title + " with " + fc + "scaling", fontsize=35)
    plt.grid(True)
    plt.show()
def plot_decision_boundary(model,xtest, ytest,name, fc):
    plt.figure(figsize=(25,15))
    #Trains model with two features
    model.fit(xtest, ytest)
    plot_decision_regions(xtest.values, ytest.ravel(),
    clf=model, legend=2)
    plt.title("Decision boundary for " + name + " with " + fc + " scaling", fontsize=35)
    plt.xlabel('Number of sexual partners',fontsize=25)
    plt.ylabel('Hormonal Contraceptives', fontsize=25)
    plt.show()


In [ ]:
#Chooses two features for decision boundary
feat_boundary = ['Number of sexual partners','Hormonal Contraceptives']
X_feature = X[feat_boundary]
X_train_feat, X_test_feat, y_train_feat, y_test_feat =train_test_split(X_feature, y, test_size = 0.2, random_state = 2021, stratify=y) 


In [ ]:
def train_model(model, X, y):
    model.fit(X, y)
    return model
def predict_model(model, X, proba=False):
    if ~proba:
        y_pred = model.predict(X)
    else:
        y_pred_proba = model.predict_proba(X)
        y_pred = np.argmax(y_pred_proba, axis=1)
    return y_pred
list_scores = []
def run_model(name, model, X_train, X_test,y_train, y_test, fc, proba=False):
    print(name)
    print(fc)
    model = train_model(model, X_train, y_train)
    y_pred = predict_model(model, X_test, proba)
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    print('accuracy: ', accuracy)
    print('recall: ',recall)
    print('precision: ', precision)
    print('f1: ', f1)
    print(classification_report(y_test, y_pred))
    plot_cm(y_test, y_pred, name, fc)
    plot_real_pred_val(y_test, y_pred, name, fc)
    plot_roc(model, X_test, y_test, name, fc)
    plot_decision_boundary(model,X_test_feat,y_test_feat, name, fc)
    plot_learning_curve(model, name, X_train, y_train, cv=3); 
    plt.show()
    list_scores.append({'Model Name': name, 'Feature Scaling':fc, 'Accuracy': accuracy, 'Recall': recall,'Precision': precision, 'F1':f1})
feature_scaling = {'Raw':(X_train_raw, X_test_raw, y_train_raw, y_test_raw),'Normalization':(X_train_norm, X_test_norm,y_train_norm, y_test_norm),'Standardization':(X_train_stand, X_test_stand,y_train_stand, y_test_stand),}



In [ ]:
#Support Vector Classifier
# Define the parameter grid for the grid search
param_grid = {
'C': [0.1, 1, 10],
'kernel': ['linear', 'poly', 'rbf'],
'gamma': ['scale', 'auto', 0.1, 1],
}
# Initialize the SVC model
model_svc = SVC(random_state=2021,
probability=True)
# Perform the grid search for each feature scaling method
for fc_name, value in feature_scaling.items():
    X_train, X_test, y_train, y_test = value
# Create GridSearchCV with the SVC model and the parameter grid
grid_search = GridSearchCV(model_svc,param_grid, cv=3, scoring='accuracy', n_jobs=-1,refit=True)
# Train and perform grid search
grid_search.fit(X_train, y_train)
# Get the best SVC model from the grid search
best_model = grid_search.best_estimator_
# Evaluate and plot the best model
run_model('SVC', model_svc, X_train, X_test,y_train, y_test, fc_name)
# Print the best hyperparameters found
print(f"Best Hyperparameters for {fc_name}:")
print(grid_search.best_params_)


In [ ]:
#Logistic Regression Classifier
# Define the parameter grid for the grid search
param_grid = {'C': [0.01, 0.1, 1, 10],'penalty': ['l1', 'l2'],'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']}
# Initialize the Logistic Regression model
logreg = LogisticRegression(max_iter=5000,random_state=2021)
# Perform the grid search for each feature scaling method
for fc_name, value in feature_scaling.items():
    X_train, X_test, y_train, y_test = value
# Create GridSearchCV with the Logistic Regression model and the parameter grid
grid_search = GridSearchCV(logreg, param_grid,cv=3, scoring='accuracy', n_jobs=-1)
# Train and perform grid search
grid_search.fit(X_train, y_train)
# Get the best Logistic Regression model from the grid search
best_model = grid_search.best_estimator_
# Evaluate and plot the best model (setting proba=True for probability prediction)
run_model('Logistic Regression', best_model, X_train, X_test, y_train, y_test, fc_name,proba=True)
# Print the best hyperparameters found
print(f"Best Hyperparameters for {fc_name}:")
print(grid_search.best_params_)


In [ ]:
#KNN Classifier
# Define the parameter grid for the grid search
param_grid = {'n_neighbors': list(range(2, 10))}
# KNN Classifier Grid Search
for fc_name, value in feature_scaling.items():
    X_train, X_test, y_train, y_test = value
# Initialize the KNN Classifier
    knn = KNeighborsClassifier()
# Create GridSearchCV with the KNN model and the parameter grid
    grid_search = GridSearchCV(knn, param_grid,cv=3, scoring='accuracy', n_jobs=-1)
    # Train and perform grid search
    grid_search.fit(X_train, y_train)
    # Get the best KNN model from the grid search
    best_model = grid_search.best_estimator_
    # Evaluate and plot the best model (setting proba=True for probability prediction)
    run_model(f'KNeighbors Classifier n_neighbors ={grid_search.best_params_["n_neighbors"]}',best_model, X_train, X_test, y_train, y_test,fc_name, proba=True)
    # Print the best hyperparameters found
    print(f"Best Hyperparameters for {fc_name}:")
    print(grid_search.best_params_)

In [ ]:
#Decision Trees Classifier 
for fc_name, value in feature_scaling.items():
    X_train, X_test, y_train, y_test = value
    # Initialize the DecisionTreeClassifier model
    dt_clf =DecisionTreeClassifier(random_state=2021)
    # Define the parameter grid for the grid search
    param_grid = {
    'max_depth': np.arange(1, 51, 1),
    'criterion': ['gini', 'entropy'],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    }
    # Create GridSearchCV with the DecisionTreeClassifier model and the parameter grid
    grid_search = GridSearchCV(dt_clf, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
    # Train and perform grid search
    grid_search.fit(X_train, y_train)
    # Get the best DecisionTreeClassifier model from the grid search
    best_model = grid_search.best_estimator_
    # Evaluate and plot the best model (setting proba=True for probability prediction)
    run_model(f'DecisionTree Classifier (Best Depth:{grid_search.best_params_["max_depth"]})',best_model, X_train, X_test, y_train, y_test,fc_name, proba=True)
    # Print the best hyperparameters found
    print(f"Best Hyperparameters for {fc_name}:")
    print(grid_search.best_params_)


In [ ]:
#Random Forest Classifier 
# Define the parameter grid for the grid search
param_grid = {
'n_estimators': [100, 200, 300],
'max_depth': [10, 20, 30, 40, 50],
'min_samples_split': [2, 5, 10],
'min_samples_leaf': [1, 2, 4]
}
# Initialize the RandomForestClassifier model
rf = RandomForestClassifier(random_state=2021)
# RandomForestClassifier Grid Search
for fc_name, value in feature_scaling.items():
    X_train, X_test, y_train, y_test = value
    # Create GridSearchCV with the RandomForestClassifier model and the parameter grid
    grid_search = GridSearchCV(rf, param_grid, cv=3, scoring='accuracy', n_jobs=-1)
    # Train and perform grid search
    grid_search.fit(X_train, y_train)
    # Get the best RandomForestClassifier model from the grid search
    best_model = grid_search.best_estimator_
    # Evaluate and plot the best model (setting proba=True for probability prediction)
    run_model(f'RandomForest Classifier (Best Estimators:{grid_search.best_params_["n_estimators"]})',best_model, X_train, X_test, y_train, y_test,fc_name, proba=True)
    # Print the best hyperparameters found
    print(f"Best Hyperparameters for {fc_name}:")
    print(grid_search.best_params_)


In [ ]:
#Gradient Boosting Classifier 
# Initialize the GradientBoostingClassifier model
gbt =GradientBoostingClassifier(random_state=2021)
# Define the parameter grid for the grid search
param_grid = {
'n_estimators': [100, 200, 300],
'max_depth': [10, 20, 30],
'subsample': [0.6, 0.8, 1.0],
'max_features': [0.2, 0.4, 0.6, 0.8, 1.0],
}
# GradientBoosting Classifier Grid Search
for fc_name, value in feature_scaling.items():
    X_train, X_test, y_train, y_test = value
    # Create GridSearchCV with the GradientBoostingClassifier model and the parameter grid
    grid_search = GridSearchCV(gbt, param_grid,cv=3, scoring='accuracy', n_jobs=-1)
    # Train and perform grid search
    grid_search.fit(X_train, y_train)
    # Get the best GradientBoostingClassifier model from the grid search
    best_model = grid_search.best_estimator_
    # Evaluate and plot the best model (setting proba=True for probability prediction)
    run_model(f'GradientBoosting Classifier (Best Estimators:{grid_search.best_params_["n_estimators"]})',best_model, X_train, X_test, y_train, y_test,fc_name, proba=True)
    # Print the best hyperparameters found
    print(f"Best Hyperparameters for {fc_name}:")
    print(grid_search.best_params_)


In [ ]:
#Extreme Gradient Boosting Classifier 
# XGBoost Classifier Grid Search
for fc_name, value in feature_scaling.items():
    X_train, X_test, y_train, y_test = value
    # Define the parameter grid for the grid search
    param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    }
    # Initialize the XGBoost classifier
    xgb = XGBClassifier(random_state=2021,use_label_encoder=False, eval_metric='mlogloss')
    # Create GridSearchCV with the XGBoost classifier and the parameter grid
    grid_search = GridSearchCV(xgb, param_grid,cv=3, scoring='accuracy', n_jobs=-1)
    # Train and perform grid search
    grid_search.fit(X_train, y_train)
    # Get the best XGBoost classifier model from the grid search
    best_model = grid_search.best_estimator_
    # Evaluate and plot the best model (setting proba=True for probability prediction)
    run_model(f'XGB Classifier (Best Estimators:{grid_search.best_params_["n_estimators"]})',best_model, X_train, X_test, y_train, y_test,fc_name, proba=True)
    # Print the best hyperparameters found
    print(f"Best Hyperparameters for {fc_name}:")
    print(grid_search.best_params_)


In [ ]:
# MLP Classifier Grid Search
for fc_name, value in feature_scaling.items():
    X_train, X_test, y_train, y_test = value
    # Define the parameter grid for the grid search
    param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50), (100, 100)],
    'activation': ['logistic', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0.0001, 0.001, 0.01],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    }
    # Initialize the MLP Classifier
    mlp = MLPClassifier(random_state=2021)
    # Create GridSearchCV with the MLP Classifier and the parameter grid
    grid_search = GridSearchCV(mlp, param_grid,cv=3, scoring='accuracy', n_jobs=-1)
    # Train and perform grid search
    grid_search.fit(X_train, y_train)
    # Get the best MLP Classifier model from the grid search
    best_model = grid_search.best_estimator_
    # Evaluate and plot the best model (setting proba=True for probability prediction)
    run_model('MLP Classifier', best_model, X_train,X_test, y_train, y_test, fc_name, proba=True)
    # Print the best hyperparameters found
    print(f"Best Hyperparameters for {fc_name}:")
    print(grid_search.best_params_)
